### Import packages

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

### Set-up

In [ ]:
infile = 'https://bitbucket.org/vishal_derive/vcu-data-mining/raw/3d740375d8d00c80e83dacbadc8b5e70cd2bfe48/data/credit_default_model_data.csv'

target = 'default payment next month'

sns.set(style='darkgrid')

### Read data

In [ ]:
df = pd.read_csv(infile)

df.head()

In [ ]:
X = df.drop([target, 'group'], axis=1)

del df

X.shape

In [ ]:
# we will take a sample for this exercise (to save time on code execution)

X = X.sample(10000, random_state=123)

X.shape

### Standardize

In [ ]:
X_scaler = StandardScaler()

# this will subtract the mean and divide by the standard deviation (for each column)
X_std = X_scaler.fit_transform(X.astype(float))

### k-means Clustering

In [ ]:
# choose a range for number of principal components
range_pca = [10, 15, 20, 25, 30, 35, 40, 45, 50]

# choose a range for number of clusters
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]

# create a dataframe to store the silhouette scores for each combination
silhouette_matrix = pd.DataFrame(index=range_pca, columns=range_n_clusters)


def perform_clus(components_to_keep_all, range_n_clusters_all):

    # for each number of principal components
    for components_to_keep in components_to_keep_all:

        pca = PCA(n_components=components_to_keep, random_state=314)

        X_pca = pca.fit_transform(X_std)

        # for each number of clusters
        for n_clus in range_n_clusters_all:

            # define the k-means model
            clusterer = KMeans(n_clusters=n_clus, random_state=314)

            # assigned a cluster number to each record
            clus_labels = clusterer.fit_predict(X_pca)

            # overall (average) Silhouette score
            silhouette_avg = silhouette_score(X_pca, clus_labels)
            
            silhouette_matrix.loc[components_to_keep, n_clus] = silhouette_avg

            print('Components:', components_to_keep, ', Clusters:', n_clus, ', Silhouette Score:', silhouette_avg)

    
perform_clus(range_pca, range_n_clusters)

In [ ]:
silhouette_matrix

In [ ]:
# create a heat map

plt.figure(figsize=(8, 6))

ax = sns.heatmap(silhouette_matrix.astype(float))
ax.set_ylim(len(silhouette_matrix), 0)
ax.set_xlabel('Number of Clusters')
ax.set_ylabel('Number of Principal Components')

plt.show()

In [ ]:
# max score within each row (max value across columns)
silhouette_matrix.max(axis=1)

10 principal components generated the highest score.

In [ ]:
# the first row (index=10) has the highest score
# let's extract that row
silhouette_matrix.loc[10, :]

In [ ]:
# find the column name where silhuette score is max

silhouette_matrix.astype(float).idxmax(axis=1)

`idxmax()` returns the index of first occurrence of maximum over requested axis.

10 principal components and 8 clusters generated the highest score.

In [ ]:
# best values that generated the highest Silhuette score

n_compoments = 10 
k = 5  # trying to keep the number of clusters small (picking 5 clusters instead of 8)

In [ ]:
pca = PCA(n_components=n_compoments, random_state=314)

X_pca = pca.fit_transform(X_std)

# define the k-means model
clusterer = KMeans(n_clusters=k, random_state=314)

# get assigned cluster numbers for each record
clus_labels = clusterer.fit_predict(X_pca)

# overall (average) Silhouette score
silhouette_avg = silhouette_score(X_pca, clus_labels)

print(silhouette_avg)

In [ ]:
clus_labels[:10]

In [ ]:
# add one to cluster numbers (so that it starts at 1 instead of 0)
X['cluster'] = clus_labels + 1

X.head()

In [ ]:
# number of records by cluster
X.cluster.value_counts()

In [ ]:
# % of records by cluster
X.cluster.value_counts() / len(X)

### Cluster Profiles

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

clus_profile = X.groupby('cluster').mean().T
clus_profile.head()

In [ ]:
clus_profile['overall'] = X.mean().T
clus_profile.head()

In [ ]:
# index 
clus_indices = clus_profile[[1, 2, 3, 4, 5]].div(clus_profile['overall'], axis=0) * 100
clus_indices.head()

Customers in cluster 3 have 33% higher balance limit as compared to the overall population. This type of information can be provided (preferably in a visual way) to provide detailed descriptions for each cluster (aka cluster profile).